<a href="https://colab.research.google.com/github/AndikaPutra509/Prediksi-Saham/blob/main/Model_Trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ta
import numpy as np
import pandas as pd
import yfinance as yf
import ta

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import TimeSeriesSplit

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

from xgboost import XGBClassifier

np.random.seed(42)

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=f9c23d989b40653fc45ddfe3f3e96548a76cf0eec2014b350653323c793ad768
  Stored in directory: /root/.cache/pip/wheels/5c/a1/5f/c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta


In [24]:
FEATURES = [
    "RSI",
    "MACD",
    "ATR",
    "Trend",
    "Volatility",
    "Volume_Z",

    "ATR_Regime",
    "Volatility_Regime",
    "Trend_Strength",
    "Liquidity_Regime",
    "Net_Pressure",

    "Efficiency",
    "Momentum_Accel",
    "Smart_Money",
    "Fake_Breakout"
]
SYMBOLS = [
    "DMND.JK",
    "ADRO.JK",
    "BBRI.JK",
    "TLKM.JK",
    "ASII.JK"
]
INTERVAL = "1h"
PERIOD = "365d"

TARGET_HORIZON = 8
RETURN_THRESHOLD = 0.01

BUY_THRESHOLD = 0.62
SELL_THRESHOLD = 0.38
# =============================
# HEDGE FUND MEMORY
# =============================
LAST_SIGNAL = None
MODEL_MEMORY = []
MEMORY_WINDOW = 30
PORTFOLIO_EQUITY = [1.0]
MAX_DRAWDOWN = 0.15
STOP_TRADING = False
MODEL_AGE = 0
MAX_MODEL_AGE = 50
USER_CAPITAL = 50_000   # bisa diganti
LOT_UNIT = 100   # aturan resmi IDX
MIN_LOT = 1

In [33]:
def fix_yfinance_columns(df):

    # hilangkan multi index
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)

    # paksa semua price column jadi Series
    for col in ["Open","High","Low","Close","Volume"]:
        if col in df.columns:
            df[col] = df[col].squeeze()

    return df

# ============================================================
# DATA DOWNLOAD
# ============================================================

def load_price():

    df = yf.download(
        SYMBOL,
        interval=INTERVAL,
        period=PERIOD,
        auto_adjust=True,
        progress=False
    )

    df = fix_yfinance_columns(df)

    return df.dropna()

# ============================================================
# FEATURE ENGINEERING
# ============================================================

def build_features(df):

    close = df["Close"]

    # ================= BASIC RETURNS =================
    df["Return"] = close.pct_change()

    # ================= MOMENTUM =================
    df["RSI"] = ta.momentum.RSIIndicator(close).rsi()

    macd = ta.trend.MACD(close)
    df["MACD"] = macd.macd_diff()

    # ================= VOLATILITY =================
    df["ATR"] = ta.volatility.AverageTrueRange(
        df["High"], df["Low"], close
    ).average_true_range()

    df["Volatility"] = df["Return"].rolling(10).std()

    # ================= TREND =================
    df["MA20"] = close.rolling(20).mean()
    df["MA50"] = close.rolling(50).mean()

    df["Trend"] = (df["MA20"] > df["MA50"]).astype(int)

    # ================= VOLUME =================
    df["Volume_Z"] = (
        df["Volume"] -
        df["Volume"].rolling(20).mean()
    ) / df["Volume"].rolling(20).std()

    # ==================================================
    # ======== ✅ HEDGE FUND UPGRADE START ============
    # ==================================================

    # --- ATR REGIME (market expansion detector)
    df["ATR_Regime"] = (
        df["ATR"] /
        df["ATR"].rolling(50).mean()
    )

    # --- VOLATILITY REGIME
    df["Volatility_Regime"] = (
        df["Volatility"] >
        df["Volatility"].rolling(50).median()
    ).astype(int)

    # --- TREND STRENGTH (institutional trend filter)
    adx = ta.trend.ADXIndicator(
        df["High"],
        df["Low"],
        close
    )

    df["ADX"] = adx.adx()

    df["Trend_Strength"] = (
        (df["ADX"]/50)
        + df["Trend"]
        + np.tanh(df["MACD"]*5)
    ) / 3

    # --- LIQUIDITY REGIME
    df["Liquidity_Regime"] = (
        df["Volume"] /
        df["Volume"].rolling(30).mean()
    )

    # --- SMART MONEY PRESSURE
    df["Buy_Pressure"] = np.where(
        df["Close"] > df["Open"],
        df["Volume"],
        0
    )

    df["Sell_Pressure"] = np.where(
        df["Close"] < df["Open"],
        df["Volume"],
        0
    )

    df["Net_Pressure"] = (
        df["Buy_Pressure"] -
        df["Sell_Pressure"]
    ) / df["Volume"]

    # ===============================
    # HEDGE FUND V2 FEATURES
    # ===============================

    # --- PRICE EFFICIENCY (trend sehat atau manipulasi)
    df["Efficiency"] = (
        abs(close.diff(10)) /
        close.diff().abs().rolling(10).sum()
    )

    # --- FAKE BREAKOUT DETECTOR
    df["Upper_Wick"] = df["High"] - df[["Close","Open"]].max(axis=1)
    df["Lower_Wick"] = df[["Close","Open"]].min(axis=1) - df["Low"]

    df["Fake_Breakout"] = (
        (df["Upper_Wick"] > 2*df["ATR"]) |
        (df["Lower_Wick"] > 2*df["ATR"])
    ).astype(int)

    # --- MOMENTUM ACCELERATION
    df["Momentum_Accel"] = df["Return"].diff()

    # --- SMART MONEY SCORE
    df["Smart_Money"] = (
        df["Net_Pressure"] *
        df["Liquidity_Regime"] *
        df["Trend_Strength"]
    )

    # ==================================================
    # ================= TARGET =========================
    # ==================================================

    future_ret = close.shift(-TARGET_HORIZON)/close - 1

    df["Target"] = np.where(
        future_ret > RETURN_THRESHOLD, 1,
        np.where(future_ret < -RETURN_THRESHOLD, 0, np.nan)
    )

    # ================= CLEANING =================
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)

    return df

def run_single_asset(symbol):

    global SYMBOL
    SYMBOL = symbol

    df = load_price()
    df = build_features(df)

    if len(df) < 200:
        raise Exception("Not enough data")

    X = df[FEATURES]
    y = df["Target"]

    # =============================
    # ENSEMBLE COMMITTEE
    # =============================
    models = build_models()

    prob_raw = ensemble_probability(
        models, X, y
    )

    # =============================
    # META CONFIDENCE
    # =============================
    confidence = model_health()

    prob = prob_raw * confidence

    # =============================
    # MACRO ADJUSTMENT
    # =============================
    prob = stress_adjustment(prob)

    regime = regime_switch(df.iloc[-1])

    atr_regime = df["ATR_Regime"].iloc[-1]

    size = position_size(prob, atr_regime)

    strategy = strategy_switch(regime)

    price = df["Close"].iloc[-1]

    signal = generate_signal(prob)

    entry, sl, tp, rr = trade_levels(df, prob)

    action = decide_action(signal, regime)

    return {
        "Symbol":symbol,
        "Price":price,
        "Prob":prob,
        "Signal":signal,
        "Action":action,
        "Entry":entry,
        "SL":sl,
        "TP":tp,
        "RR":rr,
        "PosSize":position_label(size),
        "Size":size
    }

def portfolio_scan():

    results = []

    for sym in SYMBOLS:
        try:
            res = run_single_asset(sym)
            results.append(res)

        except Exception as e:
            print(f"ERROR {sym}:", str(e))

    if len(results) == 0:
        return pd.DataFrame(
            columns=["Symbol","Prob","Size","Regime"]
        )

    return pd.DataFrame(results)

def rank_opportunities(df):

    df["Score"] = (
        df["Prob"] *
        df["Size"]
    )

    return df.sort_values(
        "Score",
        ascending=False
    )

def allocate_capital(df, capital=1.0):

    # =============================
    # SAFETY INITIALIZATION
    # =============================
    df = df.copy()

    df["Allocated_Capital"] = 0.0

    if df.empty:
        return df

    df["Edge"] = df["Prob"].apply(expected_edge)
    df["Kelly"] = df["Prob"].apply(kelly_fraction)

    df["Score"] = df["Edge"] * df["Kelly"]

    total = df["Score"].clip(lower=0).sum()

    # =============================
    # ✅ FALLBACK MODE
    # =============================
    if total <= 0:

        print("⚠️ No edge detected → Equal allocation")

        df["Allocated_Capital"] = (
            capital / len(df)
        )

        return df

    df["Allocated_Capital"] = (
        df["Score"].clip(lower=0) / total
    ) * capital

    return df

# ============================================================
# MODEL ENSEMBLE
# ============================================================

def build_models():

    models = {

        "HGB":
        HistGradientBoostingClassifier(
            learning_rate=0.03,
            max_depth=4,
            max_iter=400
        ),

        "ExtraTrees":
        ExtraTreesClassifier(
            n_estimators=600,
            max_depth=10,
            class_weight="balanced",
            n_jobs=-1
        ),

        "LogReg":
        Pipeline([
            ("imp", SimpleImputer()),
            ("sc", StandardScaler()),
            ("lr",
             LogisticRegression(
                max_iter=2000,
                class_weight="balanced"
             ))
        ]),

        "XGBoost":
        XGBClassifier(
            n_estimators=400,
            max_depth=4,
            learning_rate=0.03,
            subsample=0.9,
            colsample_bytree=0.9,
            eval_metric="logloss"
        )
    }

    return models


# ============================================================
# WALK FORWARD TRAINING
# ============================================================

def train_ensemble(X, y):

    tscv = TimeSeriesSplit(5)

    models = build_models()
    scores = {}

    for name, model in models.items():

        fold_scores = []

        for tr, val in tscv.split(X):

            model.fit(X.iloc[tr], y.iloc[tr])

            pred = model.predict(X.iloc[val])

            score = balanced_accuracy_score(
                y.iloc[val],
                pred
            )

            fold_scores.append(score)

        scores[name] = np.mean(fold_scores)

    best = max(scores, key=scores.get)

    print("\nBest model:", best)

    final_model = models[best]
    final_model.fit(X, y)

    return models

def ensemble_probability(models, X, y):

    probs = []

    for m in models.values():

        m.fit(X, y)

        p = m.predict_proba(
            X.tail(1)
        )[0][1]

        probs.append(p)

    return np.mean(probs)

# ============================================================
# SIGNAL ENGINE
# ============================================================

def decide_signal(prob):

    health = model_health()

    dynamic_buy = BUY_THRESHOLD + (0.1*(1-health))
    dynamic_sell = SELL_THRESHOLD - (0.1*(1-health))

    if prob >= dynamic_buy:
        return "BUY"

    if prob <= dynamic_sell:
        return "SELL"

    return "HOLD"


# ============================================================
# RISK ENGINE
# ============================================================

def risk_plan(price, atr):

    sl = price - 1.5 * atr
    tp = price + 2.5 * atr

    rr = (tp-price)/(price-sl)

    return sl, tp, rr

def trade_levels(df, prob):

    price = df["Close"].iloc[-1]
    atr = df["ATR"].iloc[-1]

    support = price - 1.2 * atr
    resistance = price + 2.2 * atr

    entry = price

    stop_loss = support
    take_profit = resistance

    rr = (take_profit-entry)/(entry-stop_loss)

    return entry, stop_loss, take_profit, rr

def trade_filter(row):

    if row["Fake_Breakout"] == 1:
        return False

    if row["Volatility_Regime"] == 1:
        return False

    if row["Liquidity_Regime"] < 0.7:
        return False

    return True

def position_size(prob, atr_regime):

    health = model_health()

    edge = abs(prob-0.5)*2
    vol_adj = 1/(1+atr_regime)

    size = edge * vol_adj * health

    return round(np.clip(size,0.05,1.0),2)

def update_model_memory(prob, realized_move):

    global MODEL_MEMORY

    correct = (
        (prob > 0.5 and realized_move > 0) or
        (prob <= 0.5 and realized_move <= 0)
    )

    MODEL_MEMORY.append(int(correct))

    if len(MODEL_MEMORY) > MEMORY_WINDOW:
        MODEL_MEMORY.pop(0)


def model_health():

    if len(MODEL_MEMORY) < 5:
        return 1.0

    return np.mean(MODEL_MEMORY)

def regime_switch(row):

    if row["ATR_Regime"] > 1.4:
        return "HIGH_VOL"

    if row["Trend_Strength"] > 0.6:
        return "TREND"

    return "RANGE"

def update_equity(return_today):

    global PORTFOLIO_EQUITY, STOP_TRADING

    new_equity = PORTFOLIO_EQUITY[-1] * (1 + return_today)

    PORTFOLIO_EQUITY.append(new_equity)

    peak = max(PORTFOLIO_EQUITY)

    drawdown = (peak - new_equity)/peak

    if drawdown > MAX_DRAWDOWN:
        STOP_TRADING = True

    return drawdown

def correlation_filter(symbols):

    prices = []

    for sym in symbols:
        df = yf.download(sym, period="60d",
                         interval="1d",
                         progress=False)

        prices.append(df["Close"].pct_change())

    corr = pd.concat(prices, axis=1).corr().abs()

    keep=[symbols[0]]

    for s in symbols[1:]:

        if all(corr.loc[s,k] < 0.8 for k in keep):
            keep.append(s)

    return keep

def expected_edge(prob):

    win_prob = prob
    loss_prob = 1 - prob

    avg_win = 0.02
    avg_loss = 0.01

    edge = (
        win_prob * avg_win -
        loss_prob * avg_loss
    )

    return edge

def kelly_fraction(prob):

    b = 2      # reward/risk approx
    p = prob
    q = 1 - p

    kelly = (b*p - q)/b

    return np.clip(kelly,0,0.25)

def strategy_switch(regime):

    if regime == "TREND":
        return "TREND_MODEL"

    if regime == "HIGH_VOL":
        return "DEFENSIVE"

    return "MEAN_REVERT"

def confidence_decay(prob):

    decay = np.exp(-MODEL_AGE/MAX_MODEL_AGE)

    return prob * decay

def survival_mode():

    dd = (
        max(PORTFOLIO_EQUITY) -
        PORTFOLIO_EQUITY[-1]
    ) / max(PORTFOLIO_EQUITY)

    if dd > 0.1:
        return 0.5

    if dd > 0.2:
        return 0.2

    return 1.0

def global_market_regime():

    spy = yf.download("^GSPC",
                  period="3mo",
                  auto_adjust=True,
                  progress=False)

    vix = yf.download("^VIX",
                  period="3mo",
                  auto_adjust=True,
                  progress=False)

    spy_close = spy["Close"].squeeze()
    vix_close = vix["Close"].squeeze()

    spy_ret = (
        spy_close
        .pct_change()
        .rolling(20)
        .mean()
        .iloc[-1]
    )

    vix_lvl = float(vix_close.iloc[-1])

    # ===== SAFE COMPARISON =====
    if vix_lvl > 25:
        return "CRISIS"

    if spy_ret > 0:
        return "RISK_ON"

    return "RISK_OFF"

def kill_switch():

    equity = PORTFOLIO_EQUITY[-1]
    peak = max(PORTFOLIO_EQUITY)

    dd = (peak-equity)/peak

    if dd > 0.25:
        return True

    return False

def stress_adjustment(prob):

    regime = global_market_regime()

    if regime == "CRISIS":
        return prob * 0.4

    if regime == "RISK_OFF":
        return prob * 0.7

    return prob

def cash_buffer(df):

    avg_prob = df["Prob"].mean()

    if avg_prob < 0.55:
        return 0.6   # 60% cash

    if avg_prob < 0.6:
        return 0.3

    return 0.0

def retrain_needed():

    if model_health() < 0.45:
        return True

    if MODEL_AGE > MAX_MODEL_AGE:
        return True

    return False

def portfolio_volatility_guard(df):

    if df["Prob"].std() < 0.05:
        df["Allocated_Capital"] *= 0.5

    return df

def generate_signal(prob):

    if prob >= BUY_THRESHOLD:
        return "BUY"

    if prob <= SELL_THRESHOLD:
        return "SELL"

    return "HOLD"

def capital_to_lot(capital, price):

    if price <= 0:
        return 0

    lot_price = price * LOT_UNIT

    lots = capital // lot_price

    return int(lots)

def get_idx_universe():

    # contoh universe besar (liquid IDX)
    return [
        "BBRI.JK","BMRI.JK","BBCA.JK","TLKM.JK",
        "ASII.JK","ADRO.JK","ICBP.JK","INDF.JK",
        "UNVR.JK","MDKA.JK","ANTM.JK","PGAS.JK",
        "SMGR.JK","CPIN.JK","GOTO.JK","AMRT.JK",
        "BRIS.JK","MEDC.JK","HRUM.JK","PTBA.JK",
        "ACES.JK","ERAA.JK","EXCL.JK","ISAT.JK"
    ]

def tradable_filter(symbols, capital):

    tradable = []

    for sym in symbols:

        try:
            df = yf.download(
                sym,
                period="5d",
                auto_adjust=True,
                progress=False
            )

            if df.empty:
                continue

            price = float(df["Close"].iloc[-1])

            # ✅ nilai minimum beli 1 lot IDX
            min_trade_value = price * LOT_UNIT * MIN_LOT

            # allow max 30% capital allocation
            if min_trade_value <= capital:
                tradable.append(sym)

        except:
            continue

    return tradable

def discover_opportunities(capital):

    universe = get_idx_universe()

    print("Scanning market...")

    tradable = tradable_filter(
        universe,
        capital
    )

    global SYMBOLS
    SYMBOLS = tradable

    print("Tradable assets:",len(SYMBOLS))
    print("Tradable list:", SYMBOLS)

    portfolio = portfolio_scan()

    return portfolio

def decide_action(signal, regime):

    if signal == "BUY":

        if regime == "TREND":
            return "BREAKOUT BUY"

        return "BUY PULLBACK"

    if signal == "SELL":
        return "EXIT POSITION"

    return "WAIT"

def position_label(size):

    if size > 0.7:
        return "AGGRESSIVE"

    if size > 0.4:
        return "NORMAL"

    return "SMALL"

def select_best(df, n=5):

    if df.empty:
        return df

    # hanya ambil peluang valid
    df = df[df["Prob"] > 0]

    # urutkan berdasarkan score tertinggi
    df = df.sort_values(
        "Score",
        ascending=False
    )

    return df.head(n)

# ============================================================
# MAIN PIPELINE
# ============================================================

def main():

    capital = USER_CAPITAL

    portfolio = discover_opportunities(capital)

    if portfolio.empty:
        print("No opportunities")
        return

    ranked = rank_opportunities(portfolio)

    ranked = ranked[
        ranked["Signal"] == "BUY"
    ]

    best = select_best(ranked,5)

    allocated = allocate_capital(
        best,
        capital=1.0
    )

    if "Allocated_Capital" not in allocated.columns:
        allocated["Allocated_Capital"] = 0

    allocated = allocated[
        allocated["Allocated_Capital"] > 0
    ]

    cash = cash_buffer(allocated)
    dd = (
        max(PORTFOLIO_EQUITY) -
        PORTFOLIO_EQUITY[-1]
    )/max(PORTFOLIO_EQUITY)

    print("\n========= AI TRADING TERMINAL =========")
    print("Cash Reserve :", round(cash*100,1),"%")
    print("Drawdown     :", round(dd*100,2),"%\n")

    print(
        "Symbol | Sig | Prob | Action | Lot | Capital | Entry | SL | TP | Size"
    )
    print("-"*85)

    for _,row in allocated.iterrows():

        capital_alloc = row["Allocated_Capital"] * capital

        lots = max(
            1,
            capital_to_lot(
                capital_alloc,
                row["Price"]
            )
        )

        print(
            f"{row['Symbol']:6} | "
            f"{row['Signal']:4} | "
            f"{row['Prob']:.2f} | "
            f"{row['Action'][:10]:10} | "
            f"{lots:3} | "
            f"{int(capital_alloc):7} | "
            f"{row['Entry']:.0f} | "
            f"{row['SL']:.0f} | "
            f"{row['TP']:.0f} | "
            f"{row['PosSize']}"
        )

In [34]:
if __name__ == "__main__":
    main()

Scanning market...


/tmp/ipython-input-499/2905162981.py:744: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  price = float(df["Close"].iloc[-1])
/tmp/ipython-input-499/2905162981.py:744: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  price = float(df["Close"].iloc[-1])
/tmp/ipython-input-499/2905162981.py:744: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  price = float(df["Close"].iloc[-1])
/tmp/ipython-input-499/2905162981.py:744: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  price = float(df["Close"].iloc[-1])
/tmp/ipython-input-499/2905162981.py:744: FutureWarning: Calling float on a single element Series is depreca

Tradable assets: 3
Tradable list: ['GOTO.JK', 'ACES.JK', 'ERAA.JK']

========= AI TRADING TERMINAL =========
Cash Reserve : 0.0 %
Drawdown     : 0.0 %

Symbol | Sig | Prob | Action | Lot | Capital | Entry | SL | TP | Size
-------------------------------------------------------------------------------------
